In [1]:
from cng.utils import *
from ibis import _
import os
from osgeo import gdal

con = ibis.duckdb.connect(extensions = ["spatial", "h3"])
endpoint = os.getenv("AWS_S3_ENDPOINT", "minio.carlboettiger.info")

set_secrets(con)

In [2]:
# could use bash
# ! gdalwarp -t_srs EPSG:4326 -of XYZ /vsicurl/https://data.source.coop/cboettig/mobi/species-richness-all/mobi-species-richness.tif mobi.txt

In [ ]:
# os.unlink("mobi.xyz")

def gdal_warp(input_file, output_file, dst_srs):
    ds = gdal.Open(input_file)
    src_srs = ds.GetProjection()
    warp_options = gdal.WarpOptions(dstSRS=dst_srs)
    gdal.Warp(output_file, ds, options=warp_options)

input_file ="/vsicurl/https://data.source.coop/cboettig/mobi/species-richness-all/mobi-species-richness.tif"
output_file = "mobi.xyz"
dst_srs = 'EPSG:4326'  # WGS84

gdal_warp(input_file, output_file, dst_srs)

/opt/conda/lib/python3.12/site-packages/osgeo/gdal.py:311: FutureWarning: Neither gdal.UseExceptions() nor gdal.DontUseExceptions() has been explicitly called. In GDAL 4.0, exceptions will be enabled by default.
  warnings.warn(


In [4]:

mobi = con.read_csv("mobi.xyz", 
             delim = ' ', 
             columns = {'X': 'FLOAT', 'Y': 'FLOAT', 'Z': 'INTEGER'}
            )

In [5]:


tracts_z8 = (
    con
    .read_parquet("s3://public-social-vulnerability/2022-tracts-h3-z8.parquet")
    .filter(_.STATE == "California") 
    .mutate(h8 = _.h8.lower())
)

In [ ]:

tracts_mobi = (
    tracts_z8
    .join(mobi)
    .group_by(_.COUNTY)
    .agg(richness = _.Z.mean())
    .order_by(_.richness.desc())
)  

tracts_mobi.execute()